## Enkratna podloga

### Sicret modul
```python
import sicret
```
**Modul skrivnosti se uporablja za generiranje kriptografsko močnih naključnih števil, primernih za upravljanje podatkov, kot so gesla, preverjanje pristnosti računov, varnostni žetoni in sorodne skrivnosti.**

In [ ]:
sporocilo1 = "dober dan"
sporocilo1_ascii = sporocilo1.encode("ascii")   # pretvorba v ascii
sporocilo1_utf8 = sporocilo1.encode("utf8")  # pretvorba v utf-8

print(sporocilo1_ascii)
print(sporocilo1_ascii.decode("ascii"))  # dekodiranje iz ascii

print(list(sporocilo1_ascii))   # pretvorba v seznam celih števil (bajtov)
print(bytes(sporocilo1_ascii))  # pretvorba nazaj v zaporedje bajtov
s = sporocilo1_utf8.hex()  # pretvorba v heksadecimalno predstavitev
print(bytes.fromhex(s))  # pretvorba iz heksadecimalne predstavitve nazaj v zaporedje bajtov

# Če imamo številko
num = 100
print(num.to_bytes(1, byteorder="big"))  # pretvorba v zaporedje bajtov (big-endian). Lahko uporabimo tudi "little"
# 2 pomeni število bajtov, ki jih želimo uporabiti za predstavitev številke (To izberemo glede na velikost številke 
# npr. za 100 zadostujeta 2 bajta za predstavitev do 65535 za 1 bajt pa do 255)

print(ord(sporocilo1[0]))  # pretvorba znaka v njegovo številčno (ASCII/Unicode) vrednost
print(chr(100))  # pretvorba številčne (ASCII/Unicode) vrednosti v znakS

b'dober dan'
dober dan
[100, 111, 98, 101, 114, 32, 100, 97, 110]
b'd'
100
d


## XOR

In [13]:
def xor(pt: bytes, key: bytes):
    return bytes([b^b_k for b, b_k in zip(pt, key)])

## Enkratna podloga

In [17]:
import secrets

pt = "Dober dan".encode("ascii")
key = secrets.token_bytes(len(pt))

In [18]:
def enc_OTP(pt: bytes, key: bytes):
    return xor(pt, key)

def dec_OTP(ct: bytes, key: bytes):
    return xor(ct, key)

ct = enc_OTP(pt, key)
pt = dec_OTP(ct, key)
print(key)
print(ct)
print(pt)



b"\xd0co\x1fK'\x0e\xad\xf9"
b'\x94\x0c\rz9\x07j\xcc\x97'
b'Dober dan'


## Napad (Večkratna podloga)

In [ ]:
# Večkratno poglogo napademo:
# 1. Imamo več kriptogramov, ki so bili šifrirani z isto ključno besedo
# 2. Imamo del ključa (npr. vemo, da je nek del ključa ničeln)
# 3. Imamo del izvornega besedila (npr. vemo, da se nek del izvornega besedila začne z "Dober")

# Delni napad, kjer poznamo del ključa (ničelni bajti)
def delni_napad(key: bytes, ct: bytes):
    pt_slice = bytearray()
    for k, z in zip(key, ct):
        if k == 0:
            pt_slice.append(ord("*"))
        else:
            pt_slice.append(k^z)
    return pt_slice

# Funkcija `multiple_xor(pos, cts)` na vhodu prejme pozicijo znaka v tajnopisu in seznam tajnopisov.
# Metoda nato vrne bajt ključa oz. vrednost 0, če določi, da bajta v ključu na dani poziciji ni mogoče ugotoviti.
def multiple_xor(pos: int, cts: list[bytes]) -> int:
    for z in cts:
        if pos >= len(z):
            continue
        p_key = z[pos] ^ ord(' ')
        valid = True
        for ct in cts:
            if pos >= len(ct):
                valid = False
                break
            val = ct[pos] ^ p_key
            if not (val == 32 or 97 <= val <= 122):
                valid = False
                break
        if valid:
            return p_key
    return 0

# Funkcija `guess_key(cts)` na vhodu prejme seznam tajnopisov in nato vrne seznam bajtov, ki predstavljajo ključ,
# s katerim so bili tajnopisi ustvarjeni.
# V primeri, da kakšen bajt v ključu ni mogoče ugotoviti, nastavite njegovo vrednost na 0.
def guess_key(cipher_texts: list[bytes]) -> bytearray:
    key_g = bytearray()
    max_len = max(len(c) for c in cipher_texts)
    for i in range(max_len):
        key_byte = multiple_xor(i, cipher_texts)
        key_g.append(key_byte)
    return key_g

## Gnatljivost tajnopisa

In [ ]:
email = "miha@gmail.com"

In [ ]:
# Funkcija `change_ct(ct, new_email)`, na vhodu prejme tajnopis in email naslov, vrne pa modificiran tajnopis ct'.
# Spremenite tajnopis tako, da ga bo strežnik FMTP še vedno dešifriral, a kot prejemnik ne bo naveden Bor,
# temveč email naslov, ki je podan v argumentu `new_email`.
def change_ct(ct: bytes, new_email: str) -> bytes:
    n_email_b = new_email.encode("utf8")
    if len(n_email_b) > len(ct):
        raise ValueError("New email is longer than the original ciphertext.")
    change_ct = bytearray(ct)
    for i in range(len(n_email_b)):
        change_ct[i] = ct[i] ^ (ord(email[i]) ^ n_email_b[i])
    return bytes(change_ct)